In [1]:
from port_ctrl1 import *
from CAENDesktopHighVoltagePowerSupply import CAENDesktopHighVoltagePowerSupply, OneCAENChannel
import pyvisa as visa
import time
import numpy as np
import serial
import serial.tools.list_ports 
from matplotlib import pyplot as plt

#In Jupyter notebook plots are display in static mode by default, not necessary for other interface
%matplotlib qt5

### programmable DCpower usb address for visa

#### address1 = "USB0::0x1AB1::0x0E11::DP8B242401816::INSTR" #DP832A

#### address2 = "USB0::0x1AB1::0x0E11::DP8E234100254::INSTR" #DP821A

#### address3 = "USB0::0x1AB1::0x0E11::DP8E244300633::INSTR" 

#### address4 = "USB0::0x1AB1::0x0E11::DP9A250900092::INSTR"

In [2]:
def get_com():
    Com_List = []
    plist = list(serial.tools.list_ports.comports())
    if len(plist) > 0:
        for i in range(len(plist)):
            Com_List.append(list(plist[i])[0])
    
    for com in Com_List:
        if com == "COM10":
            break
        elif Com_List[-1] != "COM12":
            raise ValueError(f'Port associate with CAEN supply is not found.')
    return com

In [3]:
# This Block build connection between PC and DC supplies
address1 = "USB0::0x1AB1::0x0E11::DP8B242401816::INSTR" #DP832A
DCport1 = Rigol_DCPort(address1)
port = get_com()
CAENport = CAENDesktopHighVoltagePowerSupply(port=port, timeout=1)

Reference DC power successfully initialized! The address is USB0::0x1AB1::0x0E11::DP8B242401816::INSTR
CAEN device connect successfully.


In [4]:
# Top module function for scanning
# Should we allow list input for interval?
def DC_scan(A, H, Arange: list, Hrange: list, Astep=None, Hstep=None, intv=None, samp=None, Achannel:str = None, Hchannel:str = None):
    '''
    Input
    A         param: Rigol_DCPort object
    H         param: CAEN control object
    Arange    param: Org and Dest of RF, a list
    Hrange    param: Org and Dest for H, a list
    Astep     param: a list of specified value (Not include org and dest, order insensitive) or just a constant
    Hstep     param: a list of specified value (Not include org and dest, order insensitive) or just a constant
    intv      param: time constant, integer or list of integers
    samp      param: # of data points, when step of each one is specified,
    Achannel  param: device channel
    Hchannel  param: device channel
    '''
    
    # Check Valid Input
    if Astep is None and Hstep is None and samp is None:
        raise ValueError(f'Please specified either the number of sampling points or step of increment for both HV voltage and RF amplitude.')
        # No input in this case
        
    elif Astep is not None and Hstep is not None and samp is not None:
        raise ValueError(f'Both specification exists, please only specify either step for both supplies or number of sampling points.')
        # Specify more than one mode
        
    elif Astep is None and Hstep is None and samp is not None:
        # Sampling with even space
        sampling = samp
        curr_q = np.linspace(Hrange[0], Hrange[1], sampling)
        
        # Check if spacing is appropriate
        if np.abs(curr_q[1] - curr_q[0]) > 40:
            # Choosing the minimal number of data points
            sampling = int(np.ceil(np.abs(Hrange[1] - Hrange[0]) / 40))
            curr_q = np.linspace(Hrange[0], Hrange[1], sampling)
        
        curr_a = np.linspace(Arange[0], Arange[1], sampling)
        if intv < 3:
            interval = 3 * np.ones(sampling)
        else:
            interval = intv * np.ones(sampling)
        
    elif Astep is not None and Hstep is not None and samp is None:
        # Sampling with specific space of specific distance
        # In this mode, only slow scan is allowed (duation > 30s)
        if type(Astep) != type(Hstep):
            raise ValueError(f'Both Astep and Hstep can either be both lists or both floating point numbers.')
        elif type(Astep) == list:
            if len(Astep) != len(Hstep):
                raise ValueError(f'When both Astep and Hstep are lists, they should have the same length. Astep has length {len(Astep)} while Hstep has length {len(Hstep)}.')
            else:
                if type(intv) is int: # In this mode, voltage persists for a custom time duration, at least 60 seconds
                    if intv < 30:
                        interval = list(30 * np.ones(sampling))
                    else:
                        interval = list(intv * np.ones(sampling))            
                elif type(intv) is list: # In this mode, duration for different voltage value can be customized, at least 60 seconds
                    interval = intv
                    for i in range(len(interval)):
                        if interval[i] < 30:
                            interval[i] = 30
                            
                curr_a = [Arange[0]] + Astep + [Arange[1]]
                curr_q = [Hrange[0]] + Hstep + [Hrange[1]]
        elif type(Astep) != list:
            # At this moment by default we set H has the same number of points as A, ignoring Hstep
            sampling = np.ceil(abs(Arange[1] - Arange[0]) / Astep)
            curr_a = np.linspace(Arange[0], Arange[1], sampling)
            curr_q = np.linspace(Hrange[0], Hrange[1], sampling)
            interval = np.max(30, intv) * np.ones(sampling)
            
    curr_v = [] # monitored value of RF amplitude
    curr_u = [] # monitored valur of DC voltage
    
    '''
    Greatest step is 25V/s
    '''
    
    '''
    # 60s needed for CAEN reach the VSET
    if type(intv) is int: # In this mode, voltage persists for a custom time duration, at least 60 seconds
        if intv < 15:
            interval = list(15 * np.ones(sampling))
        else:
            interval = list(intv * np.ones(sampling))            
    elif type(intv) is list: # In this mode, duration for different voltage value can be customized, at least 60 seconds
        interval = intv
        for i in range(len(interval)):
            if interval[i] < 15:
                interval[i] = 15
    '''
    
    # Turn on the output port for supply
    H.send_command(CMD="SET", PAR="ON", CH=Hchannel[-1])
    time.sleep(5)
    H.send_command(CMD="SET", PAR="ISET", CH=Hchannel[-1], VAL=300)
    H.send_command(CMD="SET", PAR="VSET", CH=Hchannel[-1], VAL=curr_q[0])
    if curr_q[0] <= 3000:
        time.sleep(30) # Waiting for CAEN to be set to the starting value
    else:
        time.sleep(60)
    A.ChannelOn(channel=Achannel)
    plt.ion()
    
    for i in range(sampling):
        A.set_dc_fix_value(curr_a[i], 2, channel=Achannel)
        H.send_command(CMD="SET", PAR="VSET", CH=Hchannel[-1], VAL=curr_q[i])
        #H.send_command(CMD="SET", PAR="ON", CH=Hchannel[-1])
        #time.sleep(30)
        
        for _ in range(int(interval[i])):
            # Do not try to measure output with Power supply
            try:
                Hmon = H.get_single_channel_parameter("VSET", Hchannel[-1])
                if type(Hmon) != float:
                    time.sleep(1)
                    continue
            except:
                time.sleep(1)
                continue
            
            curr_u.append(Hmon)
            curr_v.append(A.read_dc_v(channel = Achannel))
                
            plt.clf()
            plt.title("RF Amplitude vs DC Voltage")
            plt.plot(curr_v,curr_u, 'bo-')
            plt.xlabel("RF Freq Amp Q (V)")
            plt.ylabel("DC Voltage U (V)")
            plt.grid()
            plt.pause(0.001)
            plt.ioff()
            time.sleep(1)
            
    # After the entire process, close the output of both device
    H.send_command(CMD="SET", PAR="VSET", CH=0, VAL=0)
    H.send_command(CMD="SET", PAR="ISET", CH=0, VAL=0)
    time.sleep(0.5)
    H.send_command(CMD="SET", PAR="OFF", CH=Hchannel[-1])
    A.ChannelOff(channel=Achannel)
    A.reset()

In [7]:
Arange = [0,5]
Hrange = [300,500]
Astep = None #[1, 4, 2, 3] 
Hstep = None #[600, 300, 700, 500]
intv = 3
samp = 25
Achannel = "CH3"
Hchannel = "CH0"

In [11]:
DC_scan(DCport1, CAENport, Arange=Arange, Hrange=Hrange, Astep=Astep, Hstep=Hstep, intv=intv, samp=samp, Achannel=Achannel, Hchannel=Hchannel)

In [8]:
# Each time finish setting and turn on output of CAEN supply, turn off the output
CAENport.send_command(CMD="SET", PAR="VSET", CH=0, VAL=0) # Turn off Output
#CAENport.send_command(CMD="SET", PAR="ON", CH=0) # Turn off Output

In [9]:
CAENport.send_command(CMD="SET", PAR="ISET", CH=0, VAL=0)

In [10]:
CAENport.send_command(CMD="SET", PAR="OFF", CH=0) # Turn off Output, follows by 

In [ ]:
CAENport.send_command(CMD="SET", PAR="ON", CH=0)